# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# your code here
orders=pd.read_csv('Orders.zip')

print(orders.head())

   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [21]:
# your code here
##Aggregate amount spent for unique customers:
orders["CustomerID"].nunique() ## used this line to check nr of unique entries in client column
orders.groupby(["CustomerID"]).agg({'amount_spent':'sum'})

###Select Clients by quantile and label them:
Labels=["RegularClient", "VIPClient"]
qbins= pd.qcut(orders['amount_spent'],2, labels=Labels)
qbins.value_counts()

RegularClient    199112
VIPClient        198812
Name: amount_spent, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [32]:
# your code here

def Class (x):
    if x>11.8:
        return "VIPClient"
    else:
        return "RegularClient"

orders["Class"]=orders['amount_spent'].apply(Class)

orders.groupby(["Country", "Class"]).agg({'amount_spent':'count'})

amount_spent
Country              Class                      
Australia            RegularClient           113
                     VIPClient              1072
Austria              RegularClient            79
                     VIPClient               319
Bahrain              RegularClient             1
...                                          ...
United Arab Emirates VIPClient                56
United Kingdom       RegularClient        190001
                     VIPClient            164344
Unspecified          RegularClient           154
                     VIPClient                90

[72 rows x 1 columns]

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here